# Directed Graph Node Embedding (underway)

> **Graph embedding:** Given the input of a graph $\mathcal{G} = (V, E)$, and a predefined dimensionality of the embedding $d (d \ll |V|)$, the problem of graph embedding is to convert $\mathcal{G}$ into a $d$-dimensional space, in which the graph property is preserved as much as possible. The graph property can be quantified using proximity measures such as the first- and higher-order proximity. Each graph is represented as either a $d$-dimensional vector (for a whole graph) or a set of $d$-dimensional vectors with each vector representing the embedding of part of the graph (e.g., node, edge, substructure).

In Euclidean space, the feature used most after embedding is distance, which preserves the original graph property in which we are interested. In the majority situation, such similarity matric is defined based on the graph structure, like neighborhood.

In Network Representation Learning (NRL) field, the embedded vector will be used to do machine learning tasks later, and the tasks are mainly classification and clustering. Because it consider more about the structure, thus the first-order proximity is the local pairwise similarity while higher order consider about neighborhoods, and the formular here becomes $s_{ij}^{(k)} = cosine(s_{i}^{(k-1)}, s_{j}^{(k-1)})$.

Another perspective is using such embedding as a preprocessing process, which makes the information of the original graph which is preserved by the distances between objects can be easily retrieved later at the running time. In other words, it's much more like solving a regression task. For example, Euclidean Heuristic, which is used in heuristic search, uses the distances of nodes in Euclidean space to approximate the shortest path distances in orignial graph.

For both of them, there are two kinds of queries need to be finished efficiently on the embeded Euclidean space:

1. Query by example: require getting the embedding of a specific node easily. (i.e. KNN classification)
    - For representation learning, it should be able to handle the embedding of a previously unseen new nodes quickly.
    - For preprocessing, every nodes will get a pre-calculated vector as a representation. 
2. All pairs query: based on the close-formed calculation of distances in Euclidean space, this can be done quickly. (i.e. K-means clustering)

In Graph Embedding, we emphasis low-dimensional representation, which kind of related to efficient calculation. A detailed survey about it can be find [here](https://arxiv.org/abs/1709.07604).

And for graph embedding, there are two kinds of methods for similarity:

- Distance based method: and it's widely used for link prediction
- Structure based method: and it's widely used for classification

**Interlude: FastMap Algorithm and Euclidean Heuristic**

1. FastMap works on imput "Graph Constructed from Non-relational Data"
    - However, for Graph FastMap, we works on homogenerous weighted graph, homogenerous directed weighted graph underway, and stochastic graph (heterogeneous direct weighted graph) in the future.
    - The output is node embedding. 
    - However, is it merely a node embedding? Because what we use in directed graph is asymmetric distances bewteen a pair of nodes, thus it's more like a feature of edge embedding.
2. Here we exploit "semi-high-ordered proximity", which uses shortest path distance and kind of related to structure of graph but not considering the neighborhood structure that much.
3. The global structure of a graph (e.g., paths, tree, subgraph patterns) is omitted in most graph embedding techniques, and in graph FastMap we mainly uses the path information.

**Goal in this article**

How to solve the asymmetricity of directed graph and embed it into Euclidean space?

**A question**

Can structures rather than distances in Euclidean space show some properties of the original graph?

## Type One 

Some algorithms tries to get a symmetric similarity metric between nodes of directed graph, then the graph can be embedding into Euclidean space while the distances between nodes just shows the symmetric locality property.

Representaion algorithms:

[**DGE**](http://www.aaai.org/Library/IJCAI/2007/ijcai07-435.php):

The goal of this study is solving nodes classification problem on directed graph, so they proposed a method using transition probability together with the stationary distribution of Markov random walks to measure the locality property. The measure they want to minimize while embedding is $(\pi_u p(u,v)+\pi_v p(v,u))/2$, where $p(u,v) = w(u,v)/\sum_{v,u\to v}w(u,v)$. As we can see it depends on the stationary distribution $\pi_u$ and the out-link weights proportion.

**Summary:**

This is effective when solving classification and clustering tasks, and there is no need to redesign backend algorithms for specific task from embedded space.

However, what if the information we want to preserveis asymmetric per se? Which means $Dist(i,j)\ne Dist(j,i)$. Then we need a second type of embedding.

## Type Two

Using two different embedding. The consideration here is which information what to preserve? What's more, how to put the two learning embedding into one space and how they related to each other? This is what we need in DIrected Graph FastMap (DIG-FastMap).

Representaion algorithms:

#### [HOPE](https://dl.acm.org/citation.cfm?id=2939751): High-Order Proximity preserved Embedding

**Notations**: Directed Graph $G=\{V,E\}$, high-order proximity matrix is $S$, embedding matrix is $U=[U^s, U^t]$ and $U^s, U^t \in R^{N\times K}$. As we can see, here uses two embedding for directed graph.

**Problem Definition**: The high-oreder proximity here is the asymmetric transitivity, and they use **inner product** between $u_i^s$ and $u_j^t$ as **Decoder** to approximate proximity of $v_i \to v_j$.

$$\min \| S-U^s \cdot {U^t}^T \|^2_F$$

**High order proximities**: Here is a formulation which can be transform from many proximity measurements in graph, and it can facilitate the approximation of these proximities.
$$S=M_g^{-1}\cdot M_l$$

This formular then can be used to transfer into a form where $U^s$, $U^t$ can be learned through SVD (Singular Value Decomposition). And the paper also prove the upper bound of approximation. 

**Evaluation**: It mainly evaluate the method through the precision of approximation, and then from the application domain of testing its precision on directed graph reconstruction and link prediction.

**Summary**:

Here it has answer the question we proposed at the beginning: whether distances is the only metric we can use in Euclidean space? The answer is NO, and it depends on which metric we used to approximate the similarity. And here it uses the inner production to approximate the high-order proximity.

From a encoder-decoder perspective, we have a summary from a survey [paper](https://arxiv.org/abs/1709.05584):

![](./encoder-decoder.png)


Another question is about distance. Distance is what we can get from the FastMap algorithm, so, can we also uses two embedding to solve the asymmetrcity problem? And is there are difference between using distances and inner products?

Till now, a possible answer is that:
$$Inner(i, j) = v_i\cdot v_j^T = \sum_n(x_{in}x_{jn})$$
$$Dist(i, j) = \|v_i-v_j\|_F=\sqrt[F]{\sum_n(|x_{in}-x_{jn}|^F)}$$

And for $F = 2$:
$$Dist(i, j)^2 = |v_i|^2 +|v_j|^2-2\times Inner(i, j)$$

These methods are matrix-factorization approaches. Intuitively, the goal of these methods is simply to learn embeddings for each node such that the inner product between the learned embedding vectors approximates some deterministic measure of node similarity.